# Transfer Learning in Keras

In this notebook, we'll cover how to load a pre-trained model (in this case, VGGNet19) and finetune it for a new task: detecting hot dogs.

#### Load dependencies

In [3]:
from keras.applications.vgg19 import VGG19
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


#### Load the pre-trained VGG19 model

In [4]:
vgg19 = VGG19(include_top=False,
              weights='imagenet',
              input_shape=(224,224,3),
              pooling=None)

#### Freeze all the layers in the base VGGNet19 model

In [5]:
for layer in vgg19.layers:
    layer.trainable = False

#### Add custom classification layers

In [6]:
# Instantiate the sequential model and add the VGG19 model: 
model = Sequential()
model.add(vgg19)

# Add the custom layers atop the VGG19 model: 
model.add(Flatten(name='flattened'))
model.add(Dropout(0.5, name='dropout'))
model.add(Dense(2, activation='softmax', name='predictions'))

#### Compile the model for training

In [7]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#### Prepare the data for training

The dataset is available for download [here](https://www.kaggle.com/dansbecker/hot-dog-not-hot-dog/home).

In [ ]:
## N.B.: As of Nov. 1st, 2019, the above download link no longer works. 
## Instead, uncomment the two lines below to download the data: 
# ! wget -c https://www.dropbox.com/s/86r9z1kb42422up/hot-dog-not-hot-dog.tar.gz
# ! tar -xzf hot-dog-not-hot-dog.tar.gz

In [8]:
# Instantiate two image generator classes:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    data_format='channels_last',
    rotation_range=30,
    horizontal_flip=True,
    fill_mode='reflect')

valid_datagen = ImageDataGenerator(
    rescale=1.0/255,
    data_format='channels_last')

In [9]:
# Define the batch size:
batch_size=32

In [10]:
# Define the train and validation generators: 
train_generator = train_datagen.flow_from_directory(
    directory='./hot-dog-not-hot-dog/train',
    target_size=(224, 224),
    classes=['hot_dog','not_hot_dog'],
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42)

valid_generator = valid_datagen.flow_from_directory(
    directory='./hot-dog-not-hot-dog/test',
    target_size=(224, 224),
    classes=['hot_dog','not_hot_dog'],
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42)

Found 498 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


In [11]:
model.fit_generator(train_generator, steps_per_epoch=15, 
                    epochs=16, validation_data=valid_generator, 
                    validation_steps=15)

Epoch 1/16
15/15 [==============================] - 43s - loss: 0.9661 - acc: 0.5729 - val_loss: 0.5618 - val_acc: 0.7146
Epoch 2/16
15/15 [==============================] - 5s - loss: 0.7775 - acc: 0.6486 - val_loss: 0.5116 - val_acc: 0.7393
Epoch 3/16
15/15 [==============================] - 4s - loss: 0.5765 - acc: 0.7410 - val_loss: 0.6632 - val_acc: 0.6966
Epoch 4/16
15/15 [==============================] - 4s - loss: 0.4318 - acc: 0.8123 - val_loss: 0.4298 - val_acc: 0.7949
Epoch 5/16
15/15 [==============================] - 4s - loss: 0.3937 - acc: 0.8076 - val_loss: 0.4536 - val_acc: 0.7885
Epoch 6/16
15/15 [==============================] - 4s - loss: 0.3500 - acc: 0.8524 - val_loss: 0.4060 - val_acc: 0.8120
Epoch 7/16
15/15 [==============================] - 4s - loss: 0.3216 - acc: 0.8514 - val_loss: 0.5474 - val_acc: 0.7438
Epoch 8/16
15/15 [==============================] - 4s - loss: 0.2992 - acc: 0.8571 - val_loss: 0.4914 - val_acc: 0.7714
Epoch 9/16
15/15 [=============